In [67]:
# !pip install tokenizers
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.7 MB/s eta 0:00:00


In [206]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tokenizers import Tokenizer,PreTokenizedString,NormalizedString
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
torch.manual_seed(45562)

In [178]:
from tokenizers import Tokenizer, pre_tokenizers, CharBPETokenizer , decoders,  normalizers
from tokenizers.pre_tokenizers import PreTokenizer
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers.processors import TemplateProcessing
from tokenizers.models import WordPiece
from tokenizers.trainers import BpeTrainer

# Read the dataset

In [169]:
text = open('input.txt', 'r', encoding='utf-8').read()
print("length of dataset in characters: ", len(text))
# print(f"\n{text[:200]}")

length of dataset in characters:  1115394


# Build a custom tokenizer with hugging face  tokenizer API

In [147]:
pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Split("\n",'merged_with_previous'),Whitespace()])

In [146]:
pre_tokenizer.pre_tokenize_str(text[:50])

[('First Citizen:\n', (0, 15)),
 ('Before we proceed any further, hear', (15, 50))]

In [216]:
class CustomPreTokenizer:
    "A pre_tokenizer that preserves the preserves the new line character"
    def line_split(self,i: int, normalized_string: NormalizedString):
        splits = []
        for subtokens in normalized_string.split("\n"):
            for i, subtoken in enumerate(subtokens):
                if i > 0:
                    tokens.append("\n")
                if subtoken:
                    tokens.append(subtoken)
            else:
                splits.append(token)
        return splits
      
    def whitespace_split(self, i: int, normalized_string: NormalizedString):
        splits = [token for token in normalized_string.split(" ")]
        return splits

    def pre_tokenize(self, pretok: PreTokenizedString):
        pretok.split(self.line_split, )
        pretok.split(self.whitespace_split)

In [193]:
'aawdfghhf'.split()

['aawdfghhf']

In [220]:
sp_tokenizer = Tokenizer(BPE(unk_token="<unk>"))
sp_tokenizer.normalizer = normalizers.Sequence([NFD(), StripAccents()])
sp_tokenizer.pre_tokenizer = PreTokenizer.custom(CustomPreTokenizer())
sp_tokenizer.decoder = decoders.BPEDecoder(suffix="</w>")
sp_tokenizer.post_processor = TemplateProcessing(single="[CLS] $A [SEP]",
                                                 pair="[CLS] $A [SEP] $B:1 [SEP]:1",
                                                 special_tokens=[
                                                    ("[CLS]", 1),
                                                    ("[SEP]", 2),
                                                    ("[NEW-LINE]",3)])

trainer = BpeTrainer(vocab_size=500, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
sp_tokenizer.train_from_iterator(text , trainer)

Exception: ignored

In [222]:
print(sp_tokenizer.pre_tokenizer.pre_tokenize_str(text[:50]))

Exception: ignored

In [218]:
a.pre_tokenize_str(text[:50])

Exception: ignored

In [205]:
CustomPreTokenizer()

TypeError: ignored

In [153]:
tokens = []
for token in text[:50].split(' '):
      if "\n" in token:
          subtokens = token.split("\n")
          print(subtokens)
          for i, subtoken in enumerate(subtokens):
              if i > 0:
                  tokens.append("\n")
              if subtoken:
                  tokens.append(subtoken)
      else:
          tokens.append(token)


['Citizen:', 'Before']


In [152]:
tokens

['First',
 'Citizen:',
 '\n',
 'Before',
 'we',
 'proceed',
 'any',
 'further,',
 'hear']

In [123]:
toe = CharBPETokenizer()

In [113]:
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")

In [168]:
sp_tokenizer.save("sp-tokenizer.json")

In [78]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train_from_iterator(text, trainer)
tokenizer.save("sp-tokenizer.json")

In [90]:
output = tokenizer.encode(text[:50])
output.tokens

['[CLS]',
 'first',
 'citizen',
 ':',
 'before',
 'we',
 'proceed',
 'any',
 'further',
 ',',
 'hear',
 '[SEP]']

In [65]:
# tokens = nltk.regexp_tokenize(text, pattern = r'\w+|\n')

# chars = sorted(list(set(tokens)))
# # create a mapping from characters to integers
# stoi = {ch:i for i,ch in enumerate(chars)}
# itos = {i:ch for i,ch in enumerate(chars)}
# encode = lambda text: [stoi[c] for c in nltk.regexp_tokenize(text, pattern = r'\w+|\n')] 
# decode = lambda l: ' '.join([itos[i] for i in l])
# print(encode("""Advocate', 'Affection', 'Affliction'"""))
# print(decode(encode("""Advocate', 'Affection', 'Affliction'""")))

[65, 66, 67]
Advocate Affection Affliction


In [223]:
import jieba

from typing import List

from tokenizers import Tokenizer, Regex, NormalizedString, PreTokenizedString
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import PreTokenizer
from tokenizers.normalizers import Normalizer
from tokenizers.decoders import Decoder


class JiebaPreTokenizer:
    def jieba_split(self, i: int, normalized_string: NormalizedString) -> List[NormalizedString]:
        splits = []
        # we need to call `str(normalized_string)` because jieba expects a str,
        # not a NormalizedString
        for token, start, stop in jieba.tokenize(str(normalized_string)):
            splits.append(normalized_string[start:stop])

        return splits
        # We can also easily do it in one line:
        # return [normalized_string[w[1] : w[2]] for w in jieba.tokenize(str(normalized_string))]

    def odd_number_split( self, i: int, normalized_string: NormalizedString) -> List[NormalizedString]:
        # Just an odd example...
        splits = []
        last = 0
        for (i, char) in enumerate(str(normalized_string)):
            if char.isnumeric() and int(char) % 2 == 1:
                splits.append(normalized_string[last:i])
                last = i
        # Don't forget the last one
        splits.append(normalized_string[last:])
        return splits

    def pre_tokenize(self, pretok: PreTokenizedString):
        # Let's call split on the PreTokenizedString to split using `self.jieba_split`
        pretok.split(self.jieba_split)
        # Here we can call `pretok.split` multiple times if we want to apply
        # different algorithm, but we generally just need to call it once.
        pretok.split(self.odd_number_split)


class CustomDecoder:
    def decode(self, tokens: List[str]) -> str:
        return "".join(tokens)


class CustomNormalizer:
    def normalize(self, normalized: NormalizedString):
        # Most of these can be replaced by a `Sequence` combining some provided Normalizer,
        # (ie Sequence([ NFKC(), Replace(Regex("\s+"), " "), Lowercase() ])
        # and it should be the prefered way. That being said, here is an example of the kind
        # of things that can be done here:
        normalized.nfkc()
        normalized.filter(lambda char: not char.isnumeric())
        normalized.replace(Regex("\s+"), " ")
        normalized.lowercase()


# This section shows how to attach these custom components to the Tokenizer
tok = Tokenizer(BPE())
tok.normalizer = Normalizer.custom(CustomNormalizer())
tok.pre_tokenizer = PreTokenizer.custom(JiebaPreTokenizer())
tok.decoder = Decoder.custom(CustomDecoder())

input = "永和服装饰品有限公司"
print("PreTokenize:", input)
print(tok.pre_tokenizer.pre_tokenize_str(input))
# [('永和', (0, 2)), ('服装', (2, 4)), ('饰品', (4, 6)), ('有限公司', (6, 10))]

input = "112233"
print("PreTokenize:", input)
print(tok.pre_tokenizer.pre_tokenize_str(input))
# [('1', (0, 1)), ('122', (1, 4)), ('3', (4, 5)), ('3', (5, 6))]

input = "1234 ℌ𝔢𝔩𝔩𝔬    𝔱𝔥𝔢𝔯𝔢 𝓂𝓎 𝒹ℯ𝒶𝓇 𝕕𝕖𝕒𝕣    𝕗𝕣𝕚𝕖𝕟𝕕!"
print("Normalize:", input)
print(tok.normalizer.normalize_str(input))
# " hello there my dear dear friend!"

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...


PreTokenize: 永和服装饰品有限公司


Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.947 seconds.
DEBUG:jieba:Loading model cost 0.947 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


[('永和', (0, 2)), ('服装', (2, 4)), ('饰品', (4, 6)), ('有限公司', (6, 10))]
PreTokenize: 112233
[('1', (0, 1)), ('122', (1, 4)), ('3', (4, 5)), ('3', (5, 6))]
Normalize: 1234 ℌ𝔢𝔩𝔩𝔬    𝔱𝔥𝔢𝔯𝔢 𝓂𝓎 𝒹ℯ𝒶𝓇 𝕕𝕖𝕒𝕣    𝕗𝕣𝕚𝕖𝕟𝕕!
 hello there my dear dear friend!


In [226]:
list(jieba.tokenize(str(input)))

[('1234', 0, 4),
 (' ', 4, 5),
 ('ℌ', 5, 6),
 ('𝔢', 6, 7),
 ('𝔩', 7, 8),
 ('𝔩', 8, 9),
 ('𝔬', 9, 10),
 (' ', 10, 11),
 (' ', 11, 12),
 (' ', 12, 13),
 (' ', 13, 14),
 ('𝔱', 14, 15),
 ('𝔥', 15, 16),
 ('𝔢', 16, 17),
 ('𝔯', 17, 18),
 ('𝔢', 18, 19),
 (' ', 19, 20),
 ('𝓂', 20, 21),
 ('𝓎', 21, 22),
 (' ', 22, 23),
 ('𝒹', 23, 24),
 ('ℯ', 24, 25),
 ('𝒶', 25, 26),
 ('𝓇', 26, 27),
 (' ', 27, 28),
 ('𝕕', 28, 29),
 ('𝕖', 29, 30),
 ('𝕒', 30, 31),
 ('𝕣', 31, 32),
 (' ', 32, 33),
 (' ', 33, 34),
 (' ', 34, 35),
 (' ', 35, 36),
 ('𝕗', 36, 37),
 ('𝕣', 37, 38),
 ('𝕚', 38, 39),
 ('𝕖', 39, 40),
 ('𝕟', 40, 41),
 ('𝕕', 41, 42),
 ('!', 42, 43)]